In [3]:
import os
import json
import time
import requests
import random
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain import OpenAI, VectorDBQA
from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain



from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("env2.env")

def printmd(string):
    display(Markdown(string))
    
#os.makedirs("data/books/",exist_ok=True)
    

BLOB_CONTAINER_NAME = "acccsuite"
BASE_CONTAINER_URL = "https://synpasedlstore.blob.core.windows.net/" + BLOB_CONTAINER_NAME + "/"
#LOCAL_FOLDER = "./data/books/"

MODEL = "gpt-35-turbo-16k" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k

#os.makedirs(LOCAL_FOLDER,exist_ok=True)

In [4]:
from dotenv import dotenv_values, load_dotenv
# specify the name of the .env file name 
env_name = ".env2.env" # change to use your own .env file
config = dotenv_values(env_name)
load_dotenv(env_name)

True

In [5]:

# set env variables 
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"
# print(dict(config))


In [6]:
def parse_pdf(file, form_recognizer=False, formrecognizer_endpoint=None, formrecognizerkey=None, model="prebuilt-document", from_url=False, verbose=False):
    """Parses PDFs using PyPDF or Azure Document Intelligence SDK (former Azure Form Recognizer)"""
    offset = 0
    page_map = []
    if not form_recognizer:
        if verbose: print(f"Extracting text using PyPDF")
        reader = PdfReader(file)
        pages = reader.pages
        for page_num, p in enumerate(pages):
            page_text = p.extract_text()
            page_map.append((page_num, offset, page_text))
            offset += len(page_text)
    else:
        if verbose: print(f"Extracting text using Azure Document Intelligence")
        credential = AzureKeyCredential(os.environ["FORM_RECOGNIZER_KEY"])
        form_recognizer_client = DocumentAnalysisClient(endpoint=os.environ["FORM_RECOGNIZER_ENDPOINT"], credential=credential)
        
        if not from_url:
            with open(file, "rb") as filename:
                poller = form_recognizer_client.begin_analyze_document(model, document = filename)
        else:
            poller = form_recognizer_client.begin_analyze_document_from_url(model, document_url = file)
            
        form_recognizer_results = poller.result()

        for page_num, page in enumerate(form_recognizer_results.pages):
            tables_on_page = [table for table in form_recognizer_results.tables if table.bounding_regions[0].page_number == page_num + 1]

            # mark all positions of the table spans in the page
            page_offset = page.spans[0].offset
            page_length = page.spans[0].length
            table_chars = [-1]*page_length
            for table_id, table in enumerate(tables_on_page):
                for span in table.spans:
                    # replace all table spans with "table_id" in table_chars array
                    for i in range(span.length):
                        idx = span.offset - page_offset + i
                        if idx >=0 and idx < page_length:
                            table_chars[idx] = table_id

            # build page text by replacing charcters in table spans with table html
            page_text = ""
            added_tables = set()
            for idx, table_id in enumerate(table_chars):
                if table_id == -1:
                    page_text += form_recognizer_results.content[page_offset + idx]
                elif not table_id in added_tables:
                    page_text += table_to_html(tables_on_page[table_id])
                    added_tables.add(table_id)

            page_text += " "
            page_map.append((page_num, offset, page_text))
            offset += len(page_text)

    return page_map

#function to create from pdf to html
def table_to_html(table):
    table_html = "<table>"
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span > 1: cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1: cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html +="</tr>"
    table_html += "</table>"
    return table_html

#text to base64 encoding helper function
def text_to_base64(text):
    # Convert text to bytes using UTF-8 encoding
    bytes_data = text.encode('utf-8')

    # Perform Base64 encoding
    base64_encoded = base64.b64encode(bytes_data)

    # Convert the result back to a UTF-8 string representation
    base64_text = base64_encoded.decode('utf-8')

    return base64_text

In [7]:
profile_index_name = "hackchat"

### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

In [8]:
#not needed mostly TODO: we'll see this later
"""
index_payload = {
    "@odata.context": "https://hackcsi.search.windows.net/$metadata#indexes/$entity",
    "@odata.etag": "\"0x8DBF02FAD913FA2\"",
    "name": "hackchat",
    "fields": [
      {
        "name": "content",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "filepath",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "title",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "url",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "id",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "true",
        "retrievable": "true",
        "sortable": "true",
        "facetable": "false",
        "key": "true",
        "synonymMaps": []
      },
      {
        "name": "chunk_id",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "last_updated",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      }
    ],
    "scoringProfiles": [],
    "corsOptions": {
      "allowedOrigins": [
        "*"
      ],
      "maxAgeInSeconds": 300
    },
    "semantic": {
      "configurations": [
        {
          "name": "default",
          "prioritizedFields": {
            "titleField": {
              "fieldName": "title"
            },
            "prioritizedContentFields": [
              {
                "fieldName": "content"
              }
            ],
            "prioritizedKeywordsFields": []
          }
        }
      ]
    },
  }
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + profile_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(r.json())
"""

'\nindex_payload = {\n    "@odata.context": "https://hackcsi.search.windows.net/$metadata#indexes/$entity",\n    "@odata.etag": ""0x8DBF02FAD913FA2"",\n    "name": "hackchat",\n    "fields": [\n      {\n        "name": "content",\n        "type": "Edm.String",\n        "searchable": "true",\n        "filterable": "false",\n        "retrievable": "true",\n        "sortable": "false",\n        "facetable": "false",\n        "key": "false",\n        "synonymMaps": []\n      },\n      {\n        "name": "filepath",\n        "type": "Edm.String",\n        "searchable": "false",\n        "filterable": "false",\n        "retrievable": "true",\n        "sortable": "false",\n        "facetable": "false",\n        "key": "false",\n        "synonymMaps": []\n      },\n      {\n        "name": "title",\n        "type": "Edm.String",\n        "searchable": "true",\n        "filterable": "false",\n        "retrievable": "true",\n        "sortable": "false",\n        "facetable": "false",\n        "k

In [28]:
from typing import Container
#from azure.cosmosdb.table.tableservice import TableService,ListGenerator
from azure.storage.blob import BlobClient, BlobServiceClient, ContainerClient
from azure.storage.blob import ResourceTypes, AccountSasPermissions
from azure.storage.blob import generate_account_sas    
from datetime import *
import shutil

today = str(datetime.now().date())
print(today)

#================================ SOURCE ===============================
# Source Client
# connection_string = 'xxxxxxxxxxxxxxxxxxxxxx' # The connection string for the source container
# account_key = 'xxxxxxxxxxxxxxxxx' # The account key for the source container
source_container_name = 'manuals' # Name of container which has blob to be copied
connection_string = config["DOCUMENT_AZURE_STORAGE_CONNECTION_STRING"]
account_key= config["DOCUMENT_AZURE_STORAGE_CONNECTION_KEY"]

# Create client
client = BlobServiceClient.from_connection_string(connection_string) 

client = BlobServiceClient.from_connection_string(connection_string)
all_containers = client.list_containers(include_metadata=True)
# print(list(all_containers))

for container in all_containers:
    print("==========================")
    print(container['name'], container['metadata'])
    if container['name'] == 'manuals':
        print("yes", container.name)
        
        container_client = client.get_container_client(container.name)
        blobs_list = container_client.list_blobs(name_starts_with="User Manuals/")

        # print(list(blobs_list))
        
        for blob in blobs_list:
            print(blob.name)
            target_blob_name = blob.name
            print(target_blob_name)
            blob_client = client.get_blob_client(container=container['name'], blob=blob.name)
            with open(file=os.path.join(r'', blob.name), mode="wb+") as sample_blob:
                download_stream = blob_client.download_blob(max_concurrency=2)
                destfile = os.path.join(r'data/profiles', blob.name)
                sample_blob.write(download_stream.readall())
                shutil.move(blob.name, destfile)
                with open(destfile, "w") as file:
                    file.write(download_stream.readall())
                    file.close()
    


2023-11-30
manuals {}
yes manuals
[{'name': 'User Manuals/CSI Phase 1 rev 3 capital removed.pdf', 'container': 'manuals', 'snapshot': None, 'version_id': None, 'is_current_version': None, 'blob_type': <BlobType.BLOCKBLOB: 'BlockBlob'>, 'metadata': {}, 'encrypted_metadata': None, 'last_modified': datetime.datetime(2023, 11, 10, 22, 11, 8, tzinfo=datetime.timezone.utc), 'etag': '0x8DBE239F0B376DA', 'size': 1154207, 'content_range': None, 'append_blob_committed_block_count': None, 'is_append_blob_sealed': None, 'page_blob_sequence_number': None, 'server_encrypted': True, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': 'application/pdf', 'content_encoding': None, 'content_language': None, 'content_md5': bytearray(b'Q|L>e\x17\xf4\x9d\x9f\xe5\x03\r\x0cW\x98\x84'), 'content_disposition': None, 'cache_control': None}, 'lease':